<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/University_of_Los_Andes_logo.svg/1200px-University_of_Los_Andes_logo.svg.png" width="70" align="left"> 
  <h1> DEPARTAMENTO DE INGENIERIA INDUSTRIAL <br>
  Estrucuturación de Redes Sociales 202320 <br> </h1>
 
 **PROFESORES:** Felipe Montes - Andrés Felipe Usceche - Paula Cortés - Ana María Guerra - Martha Blanco 

<h1><center> Taller 1 </center></h1>
<h2><center>Diferenciar métodos para recolectar información de redes</center></h2>

Esta actividad va a estar basada en el estudio realizado por Mendoza-Palechor y de la Hoz-Manotas (2019). La base de datos suministrada recopila información de 2.111 niños provenientes de Colombia, Perú y México. Para estos niños se realizó una estimación de los niveles de obesidad basado en hábitos de comida y sus condiciones de actividad física.  

En este ejercicio vamos a comparar la diferencia entre dos metodologías de muestreo. La primera es el muestreo aleatorio y la segunda es el muestreo por bola de nieve en redes. El objetivo de esta actividad es entender la importancia del muestreo en redes para capturar las relaciones y la estructura de la red y la diferencia existente con las metodologías de muestreo estadístico que lo que buscan es representatividad muestral. 

Suponga que un grupo de la OMS le ha encargado analizar desde la perspectiva de las redes sociales la muestra del estudio de Mendoza-Palechor y de la Hoz-Manotas. Este grupo quiere que usted realice un estudio que permita determinar cuáles son los niños más influenciadores que puedan ser utilizados para compartir un programa de actividad física con sus compañeros. El problema radica en que no hay suficiente presupuesto para preguntarle a cada uno de los niños cuáles son sus amigos. Por lo tanto, le han pedido que cree una muestra a partir de la población de niños y con base en esa muestra realice el análisis de redes para determinar a los niños más influenciadores. Para esto, usted se ha propuesto seguir los siguientes pasos que le permiten cumplir el requerimiento.

In [1]:
## Librerías a usar
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
from scipy.stats import norm

In [2]:
## Cargar la red
nodos=pd.read_csv("./nodos_base.csv",index_col="id")
arcos=pd.read_csv("./arcos_base.csv")

1.	Dada la restricción presupuestal, usted sabe que lo primero que hay que hacer es determinar el tamaño de la muestra. Un aspecto relevante que le han solicitado es que la muestra que usted seleccione mantenga las proporciones de sexo de la población total. Calcule el tamaño de la muestra utilizando como referencia un nivel de confianza del 5% y un margen de error del 5%.

In [3]:
# Proporción de mujeres
m = nodos.groupby("Gender").count()["Age"]
N = m["Male"]+m["Female"]      # Poblacion Total
p = m["Female"]/N              # Proporcion girls
p             

0.49407863571766936

In [4]:
#Tamaño de la muestra
q = 1-p
e = 0.05
alpha = 0.05
Z = norm.ppf(alpha)
n = ((Z**2)*p*q)/(e**2)
n = N*n/(N+n-1)
n = math.floor(n)
n

239

2.	El grupo de la OMS no está seguro sobre la relevancia de utilizar metodologías de muestreo en redes y cree que la mejor estrategia es realizar un muestro estratificado de acuerdo al sexo. Por lo cual le han solicitado que diseñe una muestra que cumpla las especificaciones del tamaño muestral. ¿Cuál es la edad promedio de los niños y las niñas en la muestra?

In [5]:
# Promedio edad muestra mujeres
n_mujeres = int(n*p)
muestra_mujeres = nodos[nodos["Gender"]=="Female"].sample(n_mujeres)
muestra_mujeres["Age"].mean()

23.37084869491525

In [6]:
# Promedio edad muestra hombres
n_hombres = n - n_mujeres
muestra_hombres = nodos[nodos["Gender"]=="Male"].sample(n_hombres)
muestra_hombres["Age"].mean()

24.458531173553716

3.	Dado que usted ya seleccionó su muestra, ahora va a reconstruir la red de estos niños. Para esto, usted le ha preguntado a cada uno de los niños en la muestra que le mencione al a lo sumo 20 de sus amigos con los que pasa la mayoría del tiempo. Esa información ya fue recopilada y se encuentra en el archivo de arcos (por simplicidad usted tiene la información de los arcos de todos los niños). De acuerdo al grado total, ¿quiénes son los 5 niños con mayor número de conexiones?

In [7]:
## Se verifica que todos tengan menos de 20 conexiones
arcos.groupby("Source").count().sort_values("Target").max()

Target    20
dtype: int64

In [8]:
## Unir los df de muestra
muestra = pd.concat([muestra_mujeres,muestra_hombres])
muestra.sort_index()

,Gender,Age,Height,Weight
id,,,,
0,Female,21.000000,1.620000,64.000000
7,Male,22.000000,1.640000,53.000000
9,Male,22.000000,1.720000,68.000000
12,Male,22.000000,1.650000,56.000000
27,Female,23.000000,1.600000,52.000000
...,...,...,...,...
2059,Female,26.000000,1.629727,111.275646
2064,Female,20.978166,1.721057,132.054793
2088,Female,26.000000,1.633020,111.863186


In [9]:
## Sacar los arcos de la muestra
arcos_muestra = arcos[arcos["Source"].isin(muestra.index)]
arcos_muestra

,Source,Target
0,0,1184
1,0,746
2,0,1483
3,0,1515
4,0,324
...,...,...
20693,2106,111
20694,2106,155
20695,2106,869
20696,2106,589


In [10]:
## Construir Red
G1 = nx.DiGraph()
G1.add_nodes_from(muestra.index.tolist())

for index, row in arcos_muestra.iterrows():
    source = row['Source']
    target = row['Target']
    G1.add_edge(source, target)

In [11]:
# Top 5 degree
dic_grados= G1.degree()
sorted_deg = sorted(dic_grados,key=lambda item:item[1],reverse=True)
top_5_G1 = [item[0] for item in sorted_deg[:5]]
muestra.loc[top_5_G1]

,Gender,Age,Height,Weight
id,,,,
354,Female,17.000000,1.600000,59.000000
39,Female,21.000000,1.750000,88.000000
1486,Male,17.412629,1.723191,97.350366
1316,Male,39.656559,1.789992,98.021766
817,Male,22.740275,1.717288,75.948164


4.	Usted recuerda que en el curso de Estructuración de Redes Sociales le enseñaron que el muestro estadístico tiene problemas a la hora de capturar las redes de los niños. Por lo tanto, usted le ha propuesto realizar de manera paralela al grupo de la OMS un muestreo por bola de nieve. El equipo aceptó su propuesta, sin embargo, le solicitaron que el número de niños que capturara el muestro no fuera superior al tamaño de la muestra determinado previamente. ¿Cuál es la edad promedio de los niños y las niñas en la nueva muestra?

In [12]:
# Crear un diccionario para almacenar las conexiones de cada nodo
nodo_conexiones = {}

# Llenar el diccionario con las conexiones
for index, row in arcos.iterrows():
    source = row['Source']
    target = row['Target']
    
    if source in nodo_conexiones:
        nodo_conexiones[source].append(target)
    else:
        nodo_conexiones[source] = [target]

# Crear una lista de conexiones para cada nodo 
nodos['conexiones'] = nodos.index.to_series().apply(lambda x: nodo_conexiones.get(x, []))
nodos

,Gender,Age,Height,Weight,conexiones
id,,,,,
0,Female,21.000000,1.620000,64.000000,"[1184, 746, 1483, 1515, 324, 1507, 482, 366, 3..."
1,Female,21.000000,1.520000,56.000000,"[1384, 417, 991, 782, 1599, 1692, 1541, 120, 1..."
2,Male,23.000000,1.800000,77.000000,"[1872, 990, 1415, 490, 1028, 104, 636, 615, 1331]"
3,Male,27.000000,1.800000,87.000000,"[711, 630, 1790, 1876, 247, 978, 821]"
4,Male,22.000000,1.780000,89.800000,"[377, 591, 857, 1513, 37, 123, 1049, 1421, 766]"
...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,"[490, 1332, 1644, 660, 1967, 1893, 1100, 55, 1..."
2107,Female,21.982942,1.748584,133.742943,"[1583, 980, 847, 1172, 1371, 1933, 342, 377, 2..."
2108,Female,22.524036,1.752206,133.689352,"[686, 997, 1667, 1683, 162, 672, 1194, 1742, 1..."


In [13]:
def snowball():
    ultima_ola = nodos.sample(2).index.to_list()
    nodos_snowball = {i for i in ultima_ola}
    while True:
        conexiones = nodos.loc[ultima_ola, "conexiones"]
        ultima_ola = set()
        for source, lista_arcos in conexiones.iteritems():
            for target in lista_arcos:
                if target not in nodos_snowball: # Agrego un nuevo nodo
                    if len(nodos_snowball) == n: # No puedo añadir más
                        return nodos_snowball
                    nodos_snowball.add(target)
                    ultima_ola.add(target)

nodos_snowball = snowball()
arcos_snowball = arcos[arcos["Source"].isin(nodos_snowball)]
nodos_snowball = nodos[nodos.index.isin(nodos_snowball)]

In [14]:
#Edad promedio niñas
nodos_snowball[nodos_snowball["Gender"]=="Female"]["Age"].mean()

24.291357647619048

In [15]:
#Edad promedio niños
nodos_snowball[nodos_snowball["Gender"]=="Male"]["Age"].mean()

24.312301208955223

5.	Usted va a replicar el ejercicio de reconstruir la red de los niños con esta nueva muestra. De acuerdo con el grado total, ¿quiénes son los 5 niños con mayor número de conexiones?

In [16]:
## Construir Red
G2 = nx.DiGraph()
G2.add_nodes_from(nodos_snowball.index.tolist())

for index, row in arcos_snowball.iterrows():
    source = row['Source']
    target = row['Target']
    G2.add_edge(source, target)

In [17]:
# Top 5 degree
dic_grados= G2.degree()
sorted_deg = sorted(dic_grados,key=lambda item:item[1],reverse=True)
top_5_G2 = [item[0] for item in sorted_deg[:5]]
nodos_snowball.loc[top_5_G2]

,Gender,Age,Height,Weight,conexiones
id,,,,,
272,Male,18.000000,1.700000,60.000000,"[1045, 1669, 922, 503, 1933, 194, 1575, 935, 7..."
701,Male,17.282945,1.821514,59.605028,"[35, 28, 1781, 1538, 1350, 1837, 495, 1699, 15..."
895,Male,21.037514,1.636592,70.000000,"[2015, 1394, 1074, 1159, 961, 502, 1648, 376, ..."
1334,Female,22.884722,1.622787,82.000000,"[793, 260, 1675, 1679, 1243, 1641, 746, 194, 9..."
419,Male,29.000000,1.620000,89.000000,"[2042, 1212, 584, 426, 73, 103, 2060, 1313, 11..."


6.	El grupo de la OMS está complacido con los resultados, sin embargo, le han hecho la siguiente pregunta: ¿cuáles de los niños más influenciadores debería elegir: los del muestreo estadístico o los del muestreo en redes? Justifique su respuesta argumentado con base en algunas de las métricas de cada una de las redes.

In [18]:
df = {
    "Top 1": [None]*6*2,
    "Top 2": [None]*6*2,
    "Top 3": [None]*6*2,
    "Top 4": [None]*6*2,
    "Top 5": [None]*6*2,
    "Métrica": ["Degree"] * 2 + ["Centrality"] * 2 + ["Betweenness"] * 2 + 
               ["Closeness"] * 2 + ["Page Rank"] * 2 + ["Clustering"] * 2 ,
    "Muestreo": ["Estadístico", "Red"] * 6,
}

## Degree
for index, (_, degree) in enumerate(G1.degree(top_5_G1), start = 1):
    df[f"Top {index}"][0] = f"{degree:d}"
for index, (_, degree) in enumerate(G2.degree(top_5_G2), start = 1):
    df[f"Top {index}"][1] = f"{degree:d}"
    
## Centrality
c1 = nx.eigenvector_centrality(G1)
for index, node in enumerate(top_5_G1, start = 1):
    df[f"Top {index}"][2] = c1[node]
c2 = nx.eigenvector_centrality(G2)
for index, node in enumerate(top_5_G2, start = 1):
    df[f"Top {index}"][3] = c2[node]
    
## Betweenness
c1 = nx.betweenness_centrality(G1)
for index, node in enumerate(top_5_G1, start = 1):
    df[f"Top {index}"][4] = c1[node]
c2 = nx.betweenness_centrality(G2)
for index, node in enumerate(top_5_G2, start = 1):
    df[f"Top {index}"][5] = c2[node]

## Closeness
c1 = nx.closeness_centrality(G1)
for index, node in enumerate(top_5_G1, start = 1):
    df[f"Top {index}"][6] = c1[node]
c2 = nx.closeness_centrality(G2)
for index, node in enumerate(top_5_G2, start = 1):
    df[f"Top {index}"][7] = c2[node]

## Page Rank
c1 = nx.pagerank(G1)
for index, node in enumerate(top_5_G1, start = 1):
    df[f"Top {index}"][8] = c1[node]
c2 = nx.pagerank(G2)
for index, node in enumerate(top_5_G2, start = 1):
    df[f"Top {index}"][9] = c2[node]

## Clustering
c1 = nx.clustering(G1)
for index, node in enumerate(top_5_G1, start = 1):
    df[f"Top {index}"][10] = c1[node]
c2 = nx.clustering(G2)
for index, node in enumerate(top_5_G2, start = 1):
    df[f"Top {index}"][11] = c2[node]
    
pd.DataFrame(df).set_index(['Métrica', 'Muestreo'])

Top 1     Top 2     Top 3     Top 4     Top 5
Métrica     Muestreo                                                     
Degree      Estadístico        22        22        22        22        22
            Red                25        23        23        23        22
Centrality  Estadístico       0.0       0.0       0.0       0.0  0.052554
            Red          0.071535  0.060897  0.037122  0.040932  0.039618
Betweenness Estadístico   0.00012  0.000317  0.000119  0.000213   0.00018
            Red           0.00098  0.010383  0.014827  0.004951  0.003573
Closeness   Estadístico  0.002997  0.002923  0.002302  0.003401  0.003728
            Red          0.013527  0.013117  0.009635  0.011358  0.011948
Page Rank   Estadístico  0.000803  0.000656  0.000636  0.000631  0.000622
            Red          0.000826  0.000838  0.000758  0.000798  0.000753
Clustering  Estadístico         0         0         0         0         0
            Red                 0         0  0.001976  0.001976         0

7.	Exporte la red generada por el método de bola de nieve para realizar los siguintes pasos en Gephi:

In [19]:
## Exporten aquí
nx.write_gexf(G2, "graph.gexf")

1.	Deberán cargar datos de las aristas en un nuevo proyecto en Gephi. No olviden que deben identificar si la red es dirigida o no y justificar porqué.

2.	Visualicen la red bajo un algoritmo, el que ustedes prefieran, y muestren como es esta primera visualización. (Deben exportarlo mediante la herramienta de previsualización)

Para nuestro estudio, queremos detectar si existen grupos entre los niños. Para esto vamos a usar el concepto de detección de comunidades. Una de las maneras de detectar comunidades es mediante la maximización del coeficiente de modularidad utilizando el algoritmo de Louvain. Dicho algoritmo consiste en asignar los nodos a comunidades de tal forma que se maximice la función de modularidad. La función de modularidad evalúa la proporción de arcos entre nodos que se encuentran en la misma comunidad con respecto a haber cconectado los nodos de manera aleatoria. 
•	Para más información sobre el algoritmo de Louvain referirse al artículo disponible en: https://arxiv.org/pdf/0803.0476.pdf o al artículo de Wikipedia https://es.wikipedia.org/wiki/M%C3%A9todo_de_Louvain . 
•	Para mayor información acerca de la modularidad favor referirse al capítulo 7.7.1. del libro de Newman.

Con lo anterior en mente, realicen el siguiente estudio en la red.

3.	Calculen la modularidad de toda la red y muestren el reporte de modularidad. <br>
    a.	¿Cuánto es la modularidad de la red? ¿Cómo interpretarían ese valor?<br>
    b.	¿Cuántas comunidades se identifican en la red?<br>

4.	Visualicen y estudien los grupos que detectaron anteriormente. Para esto ustedes deben seguir los siguientes pasos:<br>
    a.	Coloreen los nodos a partir de las comunidades detectadas.<br>
    b.	Cambien el tamaño de los nodos a partir del grado de estos.<br>
    c.	Apliquen la distribución “Force Atlas 2” y expandan el grafo para mejorar la visualización.<br>
    d.	Muestre el grafo resultante.<br>

5.	Una vez identificadas las comunidades en la red, identifiquen cuál de las comunidades es más fuerte y cuál más débil. Para esto ustedes deben:<br>
    a.	Encontrar cuál es la comunidad que tiene un mayor numero de amistades.<br>
    b.	Identificar cuál es el grupo con mayor densidad de relaciones de amistad entre los niños.<br>


#### Realice esta actividad enteramente en Python, y una vez terminen el estudio deberán presentar sus resultados en un informe autocontenido mostrado su desarrollo y análisis. Concluida la actividad, compriman en un zip su documento, el Notebook y el archivo de Gephi de soporte.